In [ ]:
# Google Driverのマウント
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# ディスクの余分な容量を削除
!pip cache purge
!rm -rf /root/.local/share/Trash/*

In [ ]:
# ディスク容量、他を確認
!df -h /content

In [ ]:
# HAR-2way-modelのクローンコピー
!git clone -b fix/dataset_builder https://github.com/JojiOkazaki/HAR-2way-model.git
%cd HAR-2way-model
!ls

In [ ]:
# .zipの展開をしてデータセットの用意をする
!mkdir -p /content/dataset
!unzip -q "/content/drive/MyDrive/HAR-2way-model/datas/datasets.zip" -d /content/dataset
!df -h /content
!ls -R /content/dataset | head -n 20

In [ ]:
# HAR-2way-model-trainingに必要なライブラリのインストール
!pip install torch==2.5.0 torchvision pyyaml tqdm matplotlib

import torch
print(f"PyTorch Version: {torch.__version__}")
print(f"CUDA Available: {torch.cuda.is_available()}")

In [ ]:
%%writefile config_local.py

# config_local.py
from pathlib import Path
RAW_DATA_ROOT = Path("/content/dataset/").resolve()
DATASETS_ROOT = Path(RAW_DATA_ROOT / "datasets").resolve()
DATASET_NAME = "katorilab"
DATASET_ROOT = (DATASETS_ROOT / DATASET_NAME).resolve()
ARTIFACT_ROOT = Path("/content/drive/MyDrive/HAR-2way-model/artifact").resolve()

TRAIN_RATIO = 0.58
VAL_RATIO = 0.14
TEST_RATIO = 0.29
SPLIT_SEED = 42
ENABLE_GROUP_BALANCE = True
GROUP_BALANCE_ALPHA = 5.0

# ファイルパスの存在チェック
print(f"Checking paths...")
print(f"DATASET_ROOT exists: {DATASET_ROOT.exists()} ({DATASET_ROOT})")
# 例: katorilabの中身が見えるか確認
if DATASET_ROOT.exists():
    print(f"Contents of {DATASET_NAME}: {[p.name for p in DATASET_ROOT.iterdir()]}")

assert DATASET_ROOT.exists(), f"Error: {DATASET_ROOT} が見つかりません。解凍パスを確認してください。"

In [ ]:
%%writefile training/configs/train.yaml

# train.yaml
logging:
  log_dir: runs
  best_model_name: best_model.pt
  csv_name: loss_history.csv
  graph_dir: graphs
  graph_size: [6, 4]

dataset:
  name: ucf101

data:
  processed_dir: processed
  split: split01
  num_workers: 16

weighted_sampler:
  enabled: true
  alpha: 1.0

model:
  architecture:
    img:
      cnn:
        conv_channels: [3, 16, 32, 62, 128, 256]
        fc_layers: [128]
        dropouts: [0.0, 0.1, 0.2, 0.4, 0.5]
        input_size: [112, 112]
      transformer:
        d_model: 128
        nhead: 8
        num_layers: 2
        dim_ff: 256
        max_len: 32
        dropout: 0.8

    skel:
      stgcn:
        channels: [3, 64, 64, 64, 128, 128, 128]
        dropouts: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
        temporal_kernel_size: 9
        norm: batch # batch / layer

    mlp:
      layers: [256, 512, 768]
      dropouts: [0.6]

  loss_weights: [0.0, 0.0, 1.0]

training:
  batch_size: 64
  epochs: 120
  accum_steps: 1
  max_norm: 5.0 # 勾配が大きすぎないようにする。小さいと学習が遅くなり安定する。
  seed: 42
  patience: 15
  min_delta: 0.005
  recall_k: 1
  
  # proto_ce で学習（(A)）
  loss_mode: proto_ce

  # proto作成用の走査バッチ（VRAM節約で train batch と分けたい場合）
  prototype_batch_size: 64

  # valid人物判定（Trainer側で使う）
  min_valid_t: 16

  # unknown扱いしたい label_id（例: 999 を学習対象から外す）
  unknown_label_ids: [999]

  # proto_ce でも温度は使う（logitsのスケーリング）
  temperature: 0.2

optimizer:
  lr: 0.005 # 1e-3になると学習できない 0.0005 -> 0.001 (2倍)
  weight_decay: 0.05
  warmup_epochs: 3 # 学習の立ち上がりを遅くすることで序盤を安定させる。小さいと立ち上がりが速くなる。
  min_lr_ratio: 0.01 # 学習後半で学習率を小さくすることで振動を安定させる。小さいと最終学習率が小さくなる。

runtime:
  device: cuda

preprocess:
  img_aug:
    p: 0.8              # 適用確率
    jitter: 0.2         # 色調変化の強さ
    noise_std: 0.02     # ノイズ
    p_erasing: 0.3      # RandomErasing(一部隠し)の確率


In [ ]:
!nvidia-smi
print("\n\n")
!cat /proc/cpuinfo | grep "model name" | head -n 1
print("\n\n")
!free -h
print("\n\n")
!cat /etc/issue
print("\n\n")
import sys
print(f"Python version: {sys.version}")

In [ ]:
!python -m training.main --config training/configs/train.yaml